In [1]:
import os
import xml.etree.ElementTree as ET
from tqdm import tqdm

# Define paths
VOC_DIR = "Data/NEU-DET/NEU-DET/ANNOTATIONS"  # Path where your XML files are stored
YOLO_LABELS_DIR = "VOC_lab"  # Where YOLO TXT annotations will be saved
IMAGE_DIR = "Data/NEU-DET/NEU-DET/IMAGES"  # Path where your images are stored

# Define class names (Modify according to your dataset)
CLASS_NAMES = ["crazing", "inclusion","patches","pitted_surface","rolled-in_scale","scratches"]

# Create labels directory if not exists
# Create labels directory if not exists
os.makedirs(YOLO_LABELS_DIR, exist_ok=True)

# Get all XML files
xml_files = [f for f in os.listdir(VOC_DIR) if f.endswith(".xml")]

# Check for issues before conversion
empty_files = []
unknown_classes = set()
corrupted_files = []

# Function to convert VOC (Pascal) to YOLO format
def convert_voc_to_yolo(xml_file):
    try:
        tree = ET.parse(xml_file)
        root = tree.getroot()

        # Get image dimensions
        size = root.find("size")
        if size is None:
            return None  # Missing size info

        img_w = int(size.find("width").text)
        img_h = int(size.find("height").text)

        yolo_txt = ""

        objects = root.findall("object")
        if not objects:
            empty_files.append(xml_file)
            return None  # No objects in the XML

        for obj in objects:
            class_name = obj.find("name").text
            if class_name not in CLASS_NAMES:
                unknown_classes.add(class_name)
                continue  # Skip unknown classes

            class_id = CLASS_NAMES.index(class_name)
            bbox = obj.find("bndbox")
            x_min = int(bbox.find("xmin").text)
            y_min = int(bbox.find("ymin").text)
            x_max = int(bbox.find("xmax").text)
            y_max = int(bbox.find("ymax").text)

            # Convert to YOLO format (normalized values)
            x_center = ((x_min + x_max) / 2) / img_w
            y_center = ((y_min + y_max) / 2) / img_h
            width = (x_max - x_min) / img_w
            height = (y_max - y_min) / img_h

            yolo_txt += f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n"

        return yolo_txt

    except Exception as e:
        corrupted_files.append(xml_file)
        return None  # Handle corrupt XML files

# Convert all XML files
for xml_file in tqdm(xml_files, desc="Converting Annotations"):
    xml_path = os.path.join(VOC_DIR, xml_file)
    yolo_annotation = convert_voc_to_yolo(xml_path)

    if yolo_annotation:
        txt_filename = os.path.splitext(xml_file)[0] + ".txt"
        txt_path = os.path.join(YOLO_LABELS_DIR, txt_filename)
        with open(txt_path, "w") as f:
            f.write(yolo_annotation)

# Check missing labels
converted_labels = {os.path.splitext(f)[0] for f in os.listdir(YOLO_LABELS_DIR) if f.endswith(".txt")}
all_xmls = {os.path.splitext(f)[0] for f in xml_files}

missing_labels = all_xmls - converted_labels

# Print results
print(f"\n✅ Conversion Completed: XML → YOLO format!")
print(f"📝 Total XML Files: {len(xml_files)}")
print(f"⚠️ Empty XML Files: {len(empty_files)}")
print(f"❓ Unknown Classes: {unknown_classes}")
print(f"❌ Corrupted XML Files: {len(corrupted_files)}")
print(f"📌 Missing YOLO Labels: {len(missing_labels)}")
for xml_file in tqdm(xml_files, desc="Converting Annotations"):
    xml_path = os.path.join(VOC_DIR, xml_file)
    yolo_annotation = convert_voc_to_yolo(xml_path)

    if yolo_annotation:
        txt_filename = os.path.splitext(xml_file)[0] + ".txt"
        txt_path = os.path.join(YOLO_LABELS_DIR, txt_filename)
        with open(txt_path, "w") as f:
            f.write(yolo_annotation)

print("✅ Conversion Completed: XML → YOLO format!")

Converting Annotations: 100%|█████████████████████████████████████████████████████| 1800/1800 [00:12<00:00, 147.90it/s]



✅ Conversion Completed: XML → YOLO format!
📝 Total XML Files: 1800
⚠️ Empty XML Files: 0
❓ Unknown Classes: set()
❌ Corrupted XML Files: 0
📌 Missing YOLO Labels: 0


Converting Annotations: 100%|████████████████████████████████████████████████████| 1800/1800 [00:01<00:00, 1020.15it/s]

✅ Conversion Completed: XML → YOLO format!


In [2]:
import os
import shutil
import random
from tqdm import tqdm

# Define paths
DATASET_DIR =  "Data/NEU-DET/NEU-DET/IMAGES"

ANNOTATIONS_DIR = "VOC_lab"  # Folder with YOLO TXT labels

# Define new dataset split folders
TRAIN_DIR = "images/Training_data"
VAL_DIR = "images/Validation_data"
TEST_DIR = "images/Testing_data"

TRAIN_LABELS = "labels/Training_data"
VAL_LABELS = "labels/Validation_data"
TEST_LABELS = "labels/Testing_data"

# Ensure folders exist
for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR, TRAIN_LABELS, VAL_LABELS, TEST_LABELS]:
    os.makedirs(folder, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(DATASET_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))]
#random.shuffle(image_files)  # Shuffle to randomize split

# Define split numbers (Modify as needed)
num_train = 1440  # 70% for training
num_val = 180   # 20% for validation
num_test = 180  # Remaining for test

# Assign images to each set
train_images = image_files[:num_train]
val_images = image_files[num_train:num_train + num_val]
test_images = image_files[num_train + num_val:]

# Function to move images & annotations
def move_files(image_list, dest_img_folder, dest_label_folder):
    for img_file in tqdm(image_list, desc=f"Moving files to {dest_img_folder}"):
        src_img = os.path.join(DATASET_DIR, img_file)
        dst_img = os.path.join(dest_img_folder, img_file)
        shutil.move(src_img, dst_img)

        # Move corresponding label file (if exists)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label = os.path.join(ANNOTATIONS_DIR, label_file)
        dst_label = os.path.join(dest_label_folder, label_file)

        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)

# Move images and labels to respective folders
move_files(train_images, TRAIN_DIR, TRAIN_LABELS)
move_files(val_images, VAL_DIR, VAL_LABELS)
move_files(test_images, TEST_DIR, TEST_LABELS)

print("✅ Dataset split into Train, Validation, and Test sets successfully!")
print(f"🔹 Train: {num_train} images\n🔹 Val: {num_val} images\n🔹 Test: {num_test} images")

Moving files to images/Testing_data: 100%|██████████████████████████████████████████| 180/180 [00:00<00:00, 653.76it/s]

✅ Dataset split into Train, Validation, and Test sets successfully!
🔹 Train: 1440 images
🔹 Val: 180 images
🔹 Test: 180 images


In [3]:
import os
import cv2
from tqdm import tqdm

# Define paths
SOURCE_FOLDER = "Data/NEU-DET - Copy/NEU-DET/IMAGES"   # Folder containing grayscale images
DEST_FOLDER = "Data/thermal_images/NEU"      # Folder to save thermal images

# Ensure destination folder exists
os.makedirs(DEST_FOLDER, exist_ok=True)

# Get list of images
image_files = [f for f in os.listdir(SOURCE_FOLDER) if f.endswith(('.jpg', '.png', '.jpeg'))]

# Function to apply thermal effect
def convert_to_thermal(img_path, save_path):
    img = cv2.imread(img_path, cv2.IMREAD_GRAYSCALE)  # Load as grayscale
    img_colored = cv2.applyColorMap(img, cv2.COLORMAP_INFERNO)  # Apply thermal effect
    cv2.imwrite(save_path, img_colored)  # Save the thermal image

# Convert and save images
for img_file in tqdm(image_files, desc="Converting to Thermal"):
    src_path = os.path.join(SOURCE_FOLDER, img_file)
    dst_path = os.path.join(DEST_FOLDER, img_file)
    convert_to_thermal(src_path, dst_path)

print("✅ All grayscale images have been converted to thermal images!")


Converting to Thermal: 100%|███████████████████████████████████████████████████████| 1800/1800 [00:18<00:00, 96.56it/s]

✅ All grayscale images have been converted to thermal images!


In [4]:
import os
import shutil
import random
from tqdm import tqdm

# Define paths
DATASET_DIR =  "Data/thermal_images/NEU"

ANNOTATIONS_DIR = "VOC_lab"  # Folder with YOLO TXT labels

# Define new dataset split folders
TRAIN_DIR = "images/thermal/Training_data"
VAL_DIR = "images/thermal/Validation_data"
TEST_DIR = "images/thermal/Testing_data"

TRAIN_LABELS = "labels/thermal/Training_data"
VAL_LABELS = "labels/thermalValidation_data"
TEST_LABELS = "labels/thermal/Testing_data"

# Ensure folders exist
for folder in [TRAIN_DIR, VAL_DIR, TEST_DIR, TRAIN_LABELS, VAL_LABELS, TEST_LABELS]:
    os.makedirs(folder, exist_ok=True)

# Get all image files
image_files = [f for f in os.listdir(DATASET_DIR) if f.endswith(('.jpg', '.png', '.jpeg'))]
#random.shuffle(image_files)  # Shuffle to randomize split

# Define split numbers (Modify as needed)
num_train = 1440  # 70% for training
num_val = 180   # 20% for validation
num_test = 180  # Remaining for test

# Assign images to each set
train_images = image_files[:num_train]
val_images = image_files[num_train:num_train + num_val]
test_images = image_files[num_train + num_val:]

# Function to move images & annotations
def move_files(image_list, dest_img_folder, dest_label_folder):
    for img_file in tqdm(image_list, desc=f"Moving files to {dest_img_folder}"):
        src_img = os.path.join(DATASET_DIR, img_file)
        dst_img = os.path.join(dest_img_folder, img_file)
        shutil.move(src_img, dst_img)

        # Move corresponding label file (if exists)
        label_file = os.path.splitext(img_file)[0] + ".txt"
        src_label = os.path.join(ANNOTATIONS_DIR, label_file)
        dst_label = os.path.join(dest_label_folder, label_file)

        if os.path.exists(src_label):
            shutil.move(src_label, dst_label)

# Move images and labels to respective folders
move_files(train_images, TRAIN_DIR, TRAIN_LABELS)
move_files(val_images, VAL_DIR, VAL_LABELS)
move_files(test_images, TEST_DIR, TEST_LABELS)

print("✅ Dataset split into Train, Validation, and Test sets successfully!")
print(f"🔹 Train: {num_train} images\n🔹 Val: {num_val} images\n🔹 Test: {num_test} images")

Moving files to images/thermal/Testing_data: 100%|█████████████████████████████████| 180/180 [00:00<00:00, 1397.24it/s]

✅ Dataset split into Train, Validation, and Test sets successfully!
🔹 Train: 1440 images
🔹 Val: 180 images
🔹 Test: 180 images


In [5]:
import os
import glob
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Define paths
IMAGE_DIR = "images/thermal/Training_data/"
ANNOTATION_DIR = "Data/NEU-DET/NEU-DET/ANNOTATIONS"
OUTPUT_DIR = "segmented_images_thermal/"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract bounding boxes from XML
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []
    
    for obj in root.findall("object"):
        class_label = obj.find("name").text.strip()
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        objects.append((class_label, (xmin, ymin, xmax, ymax)))

    return objects
saved = 0
failed = 0
missing = 0
# Process all XML files
for xml_file in glob.glob(os.path.join(ANNOTATION_DIR, "*.xml")):
    filename = os.path.splitext(os.path.basename(xml_file))[0] + ".jpg"
    image_path = os.path.join(IMAGE_DIR, filename)
    
    if not os.path.exists(image_path):
        print(f"Warning: Image {filename} not found!")
        missing+=1
        continue

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load {filename}")
        failed+=1
        continue

    # Parse XML and extract bounding boxes
    objects = parse_annotation(xml_file)

    for idx, (class_label, (xmin, ymin, xmax, ymax)) in enumerate(objects):
        cropped_img = image[ymin:ymax, xmin:xmax]

        # Save the segmented image
        save_path = os.path.join(OUTPUT_DIR, f"{class_label}_{idx}_{filename}")
        cv2.imwrite(save_path, cropped_img)
        saved+=1
        print(f"Saved: {save_path}")


Saved: segmented_images_thermal/crazing_0_crazing_1.jpg
Saved: segmented_images_thermal/crazing_0_crazing_10.jpg
Saved: segmented_images_thermal/crazing_1_crazing_10.jpg
Saved: segmented_images_thermal/crazing_0_crazing_100.jpg
Saved: segmented_images_thermal/crazing_1_crazing_100.jpg
Saved: segmented_images_thermal/crazing_2_crazing_100.jpg
Saved: segmented_images_thermal/crazing_3_crazing_100.jpg
Saved: segmented_images_thermal/crazing_0_crazing_101.jpg
Saved: segmented_images_thermal/crazing_0_crazing_102.jpg
Saved: segmented_images_thermal/crazing_1_crazing_102.jpg
Saved: segmented_images_thermal/crazing_0_crazing_103.jpg
Saved: segmented_images_thermal/crazing_1_crazing_103.jpg
Saved: segmented_images_thermal/crazing_2_crazing_103.jpg
Saved: segmented_images_thermal/crazing_0_crazing_104.jpg
Saved: segmented_images_thermal/crazing_1_crazing_104.jpg
Saved: segmented_images_thermal/crazing_2_crazing_104.jpg
Saved: segmented_images_thermal/patches_3_crazing_104.jpg
Saved: segmented_i

In [6]:
saved

3442

In [7]:
missing

360

In [8]:
failed

0

In [9]:
import os
import glob
import cv2
import numpy as np
import xml.etree.ElementTree as ET

# Define paths
IMAGE_DIR = "images/Training_data/"
ANNOTATION_DIR = "Data/NEU-DET/NEU-DET/ANNOTATIONS"
OUTPUT_DIR = "segmented_images/"

# Create output directory if it doesn't exist
os.makedirs(OUTPUT_DIR, exist_ok=True)

# Function to extract bounding boxes from XML
def parse_annotation(xml_file):
    tree = ET.parse(xml_file)
    root = tree.getroot()
    objects = []
    
    for obj in root.findall("object"):
        class_label = obj.find("name").text.strip()
        bbox = obj.find("bndbox")
        xmin = int(bbox.find("xmin").text)
        ymin = int(bbox.find("ymin").text)
        xmax = int(bbox.find("xmax").text)
        ymax = int(bbox.find("ymax").text)
        objects.append((class_label, (xmin, ymin, xmax, ymax)))

    return objects
saved_1 = 0
failed_1 = 0
missing_1 = 0
# Process all XML files
for xml_file in glob.glob(os.path.join(ANNOTATION_DIR, "*.xml")):
    filename = os.path.splitext(os.path.basename(xml_file))[0] + ".jpg"
    image_path = os.path.join(IMAGE_DIR, filename)
    
    if not os.path.exists(image_path):
        print(f"Warning: Image {filename} not found!")
        missing_1+=1
        continue

    # Read the image
    image = cv2.imread(image_path)
    if image is None:
        print(f"Error: Could not load {filename}")
        failed_1+=1
        continue

    # Parse XML and extract bounding boxes
    objects = parse_annotation(xml_file)

    for idx, (class_label, (xmin, ymin, xmax, ymax)) in enumerate(objects):
        cropped_img = image[ymin:ymax, xmin:xmax]

        # Save the segmented image
        save_path = os.path.join(OUTPUT_DIR, f"{class_label}_{idx}_{filename}")
        cv2.imwrite(save_path, cropped_img)
        saved_1+=1
        print(f"Saved: {save_path}")


Saved: segmented_images/crazing_0_crazing_1.jpg
Saved: segmented_images/crazing_0_crazing_10.jpg
Saved: segmented_images/crazing_1_crazing_10.jpg
Saved: segmented_images/crazing_0_crazing_100.jpg
Saved: segmented_images/crazing_1_crazing_100.jpg
Saved: segmented_images/crazing_2_crazing_100.jpg
Saved: segmented_images/crazing_3_crazing_100.jpg
Saved: segmented_images/crazing_0_crazing_101.jpg
Saved: segmented_images/crazing_0_crazing_102.jpg
Saved: segmented_images/crazing_1_crazing_102.jpg
Saved: segmented_images/crazing_0_crazing_103.jpg
Saved: segmented_images/crazing_1_crazing_103.jpg
Saved: segmented_images/crazing_2_crazing_103.jpg
Saved: segmented_images/crazing_0_crazing_104.jpg
Saved: segmented_images/crazing_1_crazing_104.jpg
Saved: segmented_images/crazing_2_crazing_104.jpg
Saved: segmented_images/patches_3_crazing_104.jpg
Saved: segmented_images/crazing_0_crazing_105.jpg
Saved: segmented_images/crazing_1_crazing_105.jpg
Saved: segmented_images/crazing_2_crazing_105.jpg
Save

In [10]:
saved_1

3442

In [11]:
missing_1

360

In [12]:
failed_1

0